<a href="https://colab.research.google.com/github/rozapkk13/unet/blob/master/trainUnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install segmentation-models
!pip install onnx-tf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 6.1 MB/s eta 0:00:00


In [5]:
!pip install tensorflow-addons
!pip install tensorflow==2.15.0 tensorflow-addons==0.21.0 --upgrade --quiet

!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 6516, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 6516 (delta 325), reused 383 (delta 261), pack-reused 6051 (from 1)
Receiving objects: 100% (6516/6516), 1.98 MiB | 9.02 MiB/s, done.
Resolving deltas: 100% (5050/5050), done.
Obtaining file:///content/onnx-tensorflow
  Preparing metadata (setup.py) ... done
  Attempting uninstall: onnx-tf
    Found existing installation: onnx-tf 1.10.0
    Uninstalling onnx-tf-1.10.0:
      Successfully uninstalled onnx-tf-1.10.0
  Running setup.py develop for onnx-tf


# **Restart Runtime**

In [6]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
!pip install tf2onnx
!pip install onnx tf2onnx


In [8]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"  # Force TensorFlow Keras compatibility

import tensorflow as tf
from segmentation_models import Unet  # ✅ Import pre-trained U-Net
from tensorflow.keras.callbacks import ModelCheckpoint


Segmentation Models: using `tf.keras` framework.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import onnx
from onnx_tf.backend import prepare


ImportError: This version of TensorFlow Probability requires TensorFlow version >= 2.18; Detected an installation of version 2.15.0. Please upgrade TensorFlow to proceed.

## Train your Unet with membrane data

---
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

In [ ]:
  # model_debugger.py
# Run this first to check if the model and dataloader work correctly

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn

# Define dataset transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
def get_dataloader(batch_size=8):
    try:
        dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
        print("✅ Dataloader loaded successfully!")
        return dataloader
    except Exception as e:
        print(f"❌ Error in dataloader: {e}")
        return None

# Define a simple model
def get_model():
    try:
        model = torchvision.models.resnet18(pretrained=False)
        model.fc = nn.Linear(model.fc.in_features, 10)  # Adjust for 10 classes
        print("✅ Model loaded successfully!")
        return model
    except Exception as e:
        print(f"❌ Error in model: {e}")
        return None

# Run tests
if __name__ == "__main__":
    print("🔍 Checking Dataloader...")
    dataloader = get_dataloader()

    print("\n🔍 Checking Model...")
    model = get_model()

    # Check if one batch of data passes through the model
    if dataloader and model:
        try:
            images, labels = next(iter(dataloader))
            outputs = model(images)
            print("✅ Model forward pass successful!")
        except Exception as e:
            print(f"❌ Error in forward pass: {e}")

    print("\n✅ Debugging complete. If no errors, you can proceed to training!")


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Train with data generator

In [ ]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.onnx
import tensorflow as tf
import onnx
import tf2onnx

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
def get_dataloader(batch_size=32):
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    return trainloader

# Define a simple model
def get_model():
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)  # Adjust for 10 classes
    return model.to(device)  # Move model to correct device

# Training function
def train_model(model, dataloader, epochs=5, lr=0.001):
    model.train()  # Set model to training mode
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU if available

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {running_loss/len(dataloader)}')

    print('Training complete')

print("✅ Training complete!")




In [ ]:
print(model)
dummy_input = torch.randn(1, 3, 256, 256)  # Modify shape if needed
output = model(dummy_input)
print("Input Shape:", dummy_input.shape)
print("Output Shape:", output.shape)


In [ ]:
print("Model is defined:", "model" in locals())


In [ ]:
torch.save(model.state_dict(), 'mnist.pth')

In [ ]:
trained_model = Net()
trained_model.load_state_dict(torch.load('mnist.pth'))

dummy_input = Variable(torch.randn(1, 3, 256, 256))
torch.onnx.export(trained_model, dummy_input, "mnist.onnx")


In [ ]:
# Load the ONNX file
model = onnx.load('mnist.onnx')

# Import the ONNX model to Tensorflow
tf_rep = prepare(model)

italicized text### Train with npy file

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

In [ ]:
import torch

model_path = "trained_model.pth"

# 🔄 Load the checkpoint
checkpoint = torch.load(model_path, map_location=torch.device("cpu"))

# 🕵️‍♀️ Check if it's a full model or just state_dict
if isinstance(checkpoint, dict) and "model" in checkpoint:
    print("✅ Found saved model. Loading directly...")
    model = checkpoint["model"]  # If the whole model was saved
elif isinstance(checkpoint, dict):
    print("🔍 Found a state_dict. We need the correct model architecture!")
    print("📝 Keys in state_dict:", checkpoint.keys())
else:
    print("❌ Unsupported model format.")


### test your model and save predicted results

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from skimage import io, transform
from google.colab import files
import zipfile

# ✅ Step 1: Load TensorFlow Model
keras_model_path = "converted_model"
model = tf.saved_model.load(keras_model_path)
print("✅ TensorFlow Model Loaded Successfully!")

# ✅ Step 2: Upload & Extract Test Images
uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]
extract_path = f"/content/{zip_filename.split('.')[0]}"
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
test_path = os.path.join(extract_path, "test")
print("✅ Extracted to:", test_path)

# ✅ Step 3: Preprocessing Function
def testGenerator(test_path, target_size=(256, 256)):
    for file_name in sorted(os.listdir(test_path)):
        img_path = os.path.join(test_path, file_name)
        try:
            img = io.imread(img_path)
            if img.ndim == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            img = transform.resize(img, target_size, mode='constant', anti_aliasing=True)
            img = img / (np.max(img) + 1e-8)  # Normalize
            img = np.expand_dims(img, axis=0)
            yield img
        except Exception as e:
            print(f"🚨 Error processing {file_name}: {e}")

# ✅ Step 4: Generate Predictions
test_gen = testGenerator(test_path)
results = np.array([model.signatures["serving_default"](tf.constant(img, dtype=tf.float32))["output_0"].numpy() for img in test_gen])
results = np.nan_to_num(results)  # Fix NaNs

# ✅ Step 5: Save Predictions
save_path = os.path.join(extract_path, "predictions")
os.makedirs(save_path, exist_ok=True)

def saveResult(save_path, npyfile):
    for i, img in enumerate(npyfile):
        img = np.squeeze(img)
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)  # Normalize
        img = (img * 255).astype(np.uint8)
        img = cv2.merge([img, img, img])
        save_filename = os.path.join(save_path, f"{i}_predict.png")
        cv2.imwrite(save_filename, img)
        print(f"✅ Saved: {save_filename}")

saveResult(save_path, results)
print("🎉 Prediction Complete! Check saved images in:", save_path)


In [ ]:
import os

extract_path = 'trained_model.pth'   # Update if needed



import os
import cv2
import numpy as np
from skimage import io, transform

def testGenerator(test_path, target_size=(256, 256)):
    """
    Generator function to preprocess and yield images for testing.

    Args:
        test_path (str): Path to the test images directory.
        target_size (tuple): Target size for image resizing (default: 256x256).

    Yields:
        np.array: Preprocessed image ready for model input.
    """
    if not os.path.exists(test_path):
        raise FileNotFoundError(f"🚨 Test path not found: {test_path}")

    for file_name in sorted(os.listdir(test_path)):  # Sorted for consistency
        img_path = os.path.join(test_path, file_name)

        try:
            # Load image
            img = io.imread(img_path)

            # Convert grayscale images to RGB
            if img.ndim == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            # Resize image
            img = transform.resize(img, target_size, mode='constant', anti_aliasing=True)

            # Debugging: Check image values before normalization
            print(f"🖼 Processing {file_name} -> Min: {img.min():.4f}, Max: {img.max():.4f}, Mean: {img.mean():.4f}")

            # Normalize image (avoid NaNs by adding small constant)
            img = img / (np.max(img) + 1e-8)  

            # Add batch dimension
            img = np.expand_dims(img, axis=0)

            # Debugging: Check image values after normalization
            print(f"📊 After Normalization -> Min: {img.min():.4f}, Max: {img.max():.4f}, Mean: {img.mean():.4f}")

            yield img  # No tuple, just the image

        except Exception as e:
            print(f"🚨 Error processing {file_name}: {e}")
            continue  # Skip corrupted images
